# Email Agent with Output Guardrail and Pydantic Structure

## Project Description

This notebook implements an automated Sales Development Representative (SDR) system using the OpenAI Agents framework with the following key modifications:

### Key Modifications:

1. **Console-based Email Output**: Instead of sending emails via SendGrid, this implementation prints emails to the console in a formatted Markdown display. The `print_markdown_email` tool displays emails with proper styling using IPython's Markdown display.

2. **Email Summary Agent**: Added a new agent tool (`summary_agent`) that generates concise email summaries with **less than 10 words**. This agent uses the `print_summary` tool to output the summary.

3. **Output Guardrail for Summary Length**: Implemented an output guardrail (`guardrail_against_summary_length`) that validates the summary agent's output to ensure it contains **10 words or fewer**. If the summary exceeds 10 words, the guardrail triggers a tripwire.

4. **Pydantic Email Class Structure**: Defined an `Email` class using Pydantic's `BaseModel` with `subject` and `body` attributes. This provides type safety and structured data handling for email content throughout the agent workflow.

### Architecture:
- **Sales Manager Agent**: Orchestrates three different sales agents (professional, humorous, and concise tones) to generate email drafts, evaluates them, and selects the best one
- **Email Manager Agent**: Handles email formatting, subject generation, Markdown conversion, summary generation, and console output
- **Guardrail Agents**: Input guardrail checks for personal names in requests; output guardrail enforces summary length constraints

In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_ai_key = os.getenv("OPENAI_API_KEY")

if openai_ai_key:
    print("OpenAI API key found")
else:
    print("OpenAI API key not found")

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
OPENAI_BASE_URL = "https://api.openai.com/v1"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [ ]:
openai_client = AsyncOpenAI(base_url=OPENAI_BASE_URL, api_key=openai_ai_key)
ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [ ]:
openai_model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
ollama_model = OpenAIChatCompletionsModel(model="llama3.2", openai_client=ollama_client)

In [ ]:
sales_agent1 = Agent(name="Ollama Sales Agent", instructions=instructions1, model=ollama_model)
sales_agent2 = Agent(name="Ollama Sales Agent 2", instructions=instructions2, model=ollama_model)
sales_agent3 = Agent(name="OpenAI Sales Agent", instructions=instructions3, model=openai_model)

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [ ]:
class Email (BaseModel):
    subject: str
    body: str

In [ ]:
@function_tool
def print_markdown_email(email: Email) -> Dict[str, str]:
    display(Markdown(f"# {email.subject}\n\n{email.body}"))
    return {"status": "success"}

In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

markdown_instructions = "You can convert a text email body to an Pretty Markdown body. \
You are given a text email body which might have some markdown \
and you need to convert it to an Pretty Markdown email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

markdown_converter = Agent(name="Markdown pretty body converter", instructions=markdown_instructions, model="gpt-4o-mini")
markdown_tool = markdown_converter.as_tool(tool_name="markdown_converter", tool_description="Convert a text email body to a Pretty Markdown email body")


In [ ]:
@function_tool
def print_summary(summary: str) -> Dict[str, str]:
    """Print the summary of an email."""
    print("SUMMARY:")
    print(summary)
    return {"status": "success"}

In [ ]:
summary_tools = [print_summary]

In [ ]:
class SummaryWordsLength(BaseModel):
    has_more_than_10_words: bool
    summary: str

guardrail_summary_length_agent = Agent(
    name = "Guardrail Summary Length",
    instructions = "You are a guardrail that checks if the summary has more than 10 words",
    output_type=SummaryWordsLength,
    model = "gpt-4o-mini"
)

In [ ]:
@output_guardrail
async def guardrail_against_summary_length(ctx, agent, message):
    result = await Runner.run(guardrail_summary_length_agent, message, context=ctx.context)
    has_more_than_10_words = result.final_output.has_more_than_10_words
    return GuardrailFunctionOutput(output_info={"has_more_than_10_words": result.final_output}, tripwire_triggered=has_more_than_10_words)

In [ ]:
# summary_agent_instructions = "Summarize the provided email text in 10 words or fewer. Be concise and capture the main point only.\
#     After that, use the print_summary tool to print the summary."

summary_agent_instructions = "Summarize the provided email text in at least 11 words or more. Be concise and capture the main point only.\
    After that, use the print_summary tool to print the summary."

summary_agent = Agent(
    name="Summary Agent",
    instructions=summary_agent_instructions,
    tools=[print_summary],
    model="gpt-4o-mini",
    output_guardrails=[guardrail_against_summary_length])

summary_agent_tool = summary_agent.as_tool(tool_name="summary_agent", tool_description="Summarize the provided email text")

In [ ]:
email_tools = [subject_tool, markdown_tool, print_markdown_email, summary_agent_tool]

In [ ]:
instructions ="You are an email formatter and printer. You receive the body of an email to be printed. \
You first use the subject_writer tool to write a subject for the email, then use the markdown_converter tool to convert the body to Markdown. \
then you use the print_markdown_email tool to print the email with the subject and Markdown body. \
finally you use the summary_agent tool to summarize the email"

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools = email_tools,
    model = "gpt-4o-mini",
    handoff_description="Convert an email to Markdown and print it")

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Printer: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and printing.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

In [ ]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent(
    name = "Name Check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [ ]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Sales Department"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)